This Colab shows how to load the provided `.npz` file with rank-$49$ factorizations of $\boldsymbol{\mathscr{T}}_4$, and how to compute the invariants $\mathscr{R}$ and $\mathscr{K}$ in order to demonstrate that these factorizations are mutually nonequivalent.

- Copyright 2022 DeepMind Technologies Limited
- All software is licensed under the Apache License, Version 2.0 (Apache 2.0); you may not use this file except in compliance with the Apache 2.0 license. You may obtain a copy of the Apache 2.0 license at: https://www.apache.org/licenses/LICENSE-2.0
- All other materials are licensed under the Creative Commons Attribution 4.0 International License (CC-BY).  You may obtain a copy of the CC-BY license at: https://creativecommons.org/licenses/by/4.0/legalcode
- Unless required by applicable law or agreed to in writing, all software and materials distributed here under the Apache 2.0 or CC-BY licenses are distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the licenses for the specific language governing permissions and limitations under those licenses.
- This is not an official Google product.

In [ ]:
from google.colab import files
from typing import Hashable, List, Optional, Tuple

import numpy as np

In [ ]:
# Upload the provided `.npz` file containing the factorizations.
chosen_file = list(files.upload().keys())[0]
factorizations = np.load(open(chosen_file, 'rb'))['factorizations']
print(factorizations.shape)

**Compute basic properties of the factorizations**

In [ ]:
entries = list(np.unique(factorizations))
print('Factors include {} as entries.'.format(', '.join(map(str, entries))))

In [ ]:
nonzeros = np.count_nonzero(factorizations, axis=(-3, -2, -1))
print('Nonzeros in a factorization range from {} to {}.'.format(
    np.min(nonzeros), np.max(nonzeros)))

**Code to compute the invariants $\mathscr{R}$ and $\mathscr{K}$**

In code comments we write `S` for the tensor size (equals $16$ for $\boldsymbol{\mathscr{T}}_4$) and `R` for the rank (equals $49$ in the provided `.npz` file).

In [ ]:
def _matricize_factorization(factorization: np.ndarray) -> np.ndarray:
  """Transforms [R, 3, S] `factorization` into [R, 3, sqrt S, sqrt S]."""
  rank, _, tensor_size = factorization.shape
  matrix_size = int(np.sqrt(tensor_size))
  return factorization.reshape((rank, 3, matrix_size, matrix_size))


def compute_invariant_r(factorization: np.ndarray) -> Hashable:
  """Returns the matrix rank invariant of `factorization`.

  The matrix rank invariant of a factorization {U_r, V_r, W_r}_{r=1}^R is
      { { rank(U_r), rank(V_r), rank(W_r) } }_{r=1}^R
  where {.} denotes an unordered tuple, and rank(X_r) is the matrix rank of the
  factor X_r when seen as a square matrix. See Supplementary Information of the
  paper for more details.

  Args:
    factorization: [R, 3, S] array representing {U_r, V_r, W_r}_{r=1}^R
  Returns:
    The matrix rank invariant of `factorization`. The unordered tuples are
    returned in a canonical form (sorted).
  """
  matricized = _matricize_factorization(factorization)  # [R, 3, sqrt S, sqrt S]
  ranks = np.linalg.matrix_rank(matricized)  # [R, 3]
  return tuple(sorted(tuple(sorted(factor_ranks)) for factor_ranks in ranks))


def _action_abc(a: np.ndarray, b: np.ndarray, c: np.ndarray,
                matricized_factorization: np.ndarray) -> np.ndarray:
  """Applies the (A, B, C) action to `matricized_factorization`."""
  inv_a = np.linalg.inv(a)  # [sqrt(S), sqrt(S)]
  inv_b = np.linalg.inv(b)  # [sqrt(S), sqrt(S)]
  inv_c = np.linalg.inv(c)  # [sqrt(S), sqrt(S)]
  return np.array([[a @ u @ inv_b, b @ v @ inv_c, c @ w @ inv_a]
                   for u, v, w in matricized_factorization])

def _phi(matricized_factorization: np.ndarray) -> Optional[np.ndarray]:
  """Canonicalizes the factorization to reveal a (I, I, I) factor.

  This corresponds to the map phi described in the Supplementary Information of
  the paper.

  Args:
    matricized_factorization: [R, 3, sqrt S, sqrt S] array
  Returns:
    - `None` if the factorization does not have exactly one factor such that
      U_r V_r W_r equals the identity matrix.
    - Otherwise, an [R, 3, sqrt S, sqrt S] array representing the matricized
      factorization after applying the (A, B, C) action that transforms the
      factor (U_r, V_r, W_r) satisfying U_r V_r W_r = I into (I, I, I).
  """
  matrix_size = matricized_factorization.shape[-1]

  # Check if there is exactly one factor with U_r V_r W_r = I.
  u, v, w = np.moveaxis(matricized_factorization, 1, 0)
  is_uvw_identity = np.all(u @ v @ w == np.eye(matrix_size), axis=(1, 2))  # [R]
  if np.sum(is_uvw_identity) != 1:
    return None
  identity_index = np.argmax(is_uvw_identity)  # []

  # Apply (A, B, C) action that transforms this factor into (I, I, I).
  u1, v1, w1 = matricized_factorization[identity_index]
  matrix_a = np.eye(matrix_size)  # [sqrt(S), sqrt(S)]
  matrix_c = np.linalg.inv(w1)  # [sqrt(S), sqrt(S)]
  matrix_b = matrix_c @ np.linalg.inv(v1)  # [sqrt(S), sqrt(S)]
  return _action_abc(matrix_a, matrix_b, matrix_c, matricized_factorization)

def compute_invariant_k(factorization: np.ndarray) -> Hashable:
  """Computes the more granular invariant of `factorization`, or returns `None`.

  See the Supplementary Information of the paper for a detailed description of
  this invariant, and a proof of correctness.

  Args:
    factorization: [R, 3, S] array representing {U_r, V_r, W_r}_{r=1}^R
  Returns:
    The invariant K of `factorization`. For factorizations where this invariant
    is not applicable, a constant dummy value is returned (so that all these
    factorizations are considered indistinguishable using this invariant).
  """
  matricized = _matricize_factorization(factorization)  # [R, 3, sqrt S, sqrt S]
  canonicalized = _phi(matricized)  # [R, 3, sqrt S, sqrt S]
  if canonicalized is None:
    return ()  # Dummy value when the invariant is not applicable.
  return frozenset(
      # `np.poly` computes the coefficients of the characteristic polynomial
      # of the Kronecker product u x v x w.
      tuple(np.round(np.poly(np.kron(np.kron(u, v), w))).astype(np.int32))
      for u, v, w in canonicalized)

**Run invariant computations on the loaded factorizations**

This can take up to 10 minutes to complete, but there will be intermediate outputs.

In [ ]:
invariants: List[Tuple[Hashable, Hashable]] = []
for fi, current_factorization in enumerate(factorizations):
  invariant_r = compute_invariant_r(current_factorization)
  invariant_k = compute_invariant_k(current_factorization)
  invariants.append((invariant_r, invariant_k))
  if (fi + 1) % 500 == 0 or (fi + 1) == len(factorizations):
    print('After processing {}/{} factorizations: unique invariant values '
          '(number of nonequivalent factorizations): {}'.format(
              fi + 1, len(factorizations), len(set(invariants))))